# Module 02: Text Representation (Classical)

**Bag of Words, TF-IDF, and N-grams - Your Essential Baselines**

---

## 1. Objectives

- ✅ Understand Bag of Words representation and its limitations
- ✅ Master TF-IDF from scratch with mathematical derivation
- ✅ Implement N-grams for capturing local context
- ✅ Build a complete text classification baseline

## 2. Prerequisites

- [Module 01: Text Preprocessing](../01_text_preprocessing/01_text_preprocessing.ipynb)

## 3. Intuition & Motivation

### The Problem
Machine learning models need **numbers**, not text. How do we convert:

`"I love this movie"` → `[0.2, 0.5, 0.1, ...]`

### Classical Approaches

| Method | Idea | Captures Meaning? |
|--------|------|------------------|
| Bag of Words | Count words | ❌ No semantics |
| TF-IDF | Weight by importance | ❌ No semantics |
| N-grams | Local word patterns | 🔶 Some context |

> **These are your BASELINES** - always try them first!

In [ ]:
# Setup
import numpy as np
from collections import Counter
from typing import List, Dict
import math

# Sample corpus for examples
corpus = [
    "I love this movie it is amazing",
    "This movie is terrible I hate it",
    "Great film with amazing acting",
    "Worst movie ever seen terrible acting"
]
labels = [1, 0, 1, 0]  # 1=positive, 0=negative

## 4. Mathematical Foundation

### 4.1 Bag of Words (BoW)

**Idea**: Represent text as word counts, ignoring order.

For vocabulary $V = \{w_1, w_2, ..., w_n\}$ and document $d$:

$$\text{BoW}(d) = [count(w_1, d), count(w_2, d), ..., count(w_n, d)]$$

### 4.2 TF-IDF

**Idea**: Words that appear frequently in a document but rarely across documents are more informative.

**Term Frequency (TF)**:
$$TF(t, d) = \frac{\text{count of } t \text{ in } d}{\text{total words in } d}$$

**Inverse Document Frequency (IDF)**:
$$IDF(t) = \log\left(\frac{N}{df(t)}\right)$$

Where:
- $N$ = total number of documents
- $df(t)$ = number of documents containing term $t$

**TF-IDF**:
$$\text{TF-IDF}(t, d) = TF(t, d) \times IDF(t)$$

## 5. Implementation from Scratch

### 5.1 Bag of Words

In [ ]:
class BagOfWords:
    """Bag of Words vectorizer from scratch."""
    
    def __init__(self):
        self.vocab = {}  # word -> index
        self.vocab_size = 0
    
    def fit(self, documents: List[str]):
        """Build vocabulary from documents."""
        all_words = set()
        for doc in documents:
            all_words.update(doc.lower().split())
        
        self.vocab = {word: idx for idx, word in enumerate(sorted(all_words))}
        self.vocab_size = len(self.vocab)
        print(f"Vocabulary size: {self.vocab_size}")
        return self
    
    def transform(self, documents: List[str]) -> np.ndarray:
        """Convert documents to BoW vectors."""
        vectors = np.zeros((len(documents), self.vocab_size))
        
        for i, doc in enumerate(documents):
            word_counts = Counter(doc.lower().split())
            for word, count in word_counts.items():
                if word in self.vocab:
                    vectors[i, self.vocab[word]] = count
        return vectors
    
    def fit_transform(self, documents: List[str]) -> np.ndarray:
        return self.fit(documents).transform(documents)

# Test BoW
bow = BagOfWords()
X_bow = bow.fit_transform(corpus)

print(f"\nShape: {X_bow.shape}")
print(f"Vocabulary: {list(bow.vocab.keys())[:10]}...")
print(f"\nFirst document vector (non-zero): {[(list(bow.vocab.keys())[i], int(X_bow[0,i])) for i in np.nonzero(X_bow[0])[0]]}")

### 5.2 TF-IDF from Scratch

In [ ]:
class TfidfVectorizer:
    """TF-IDF vectorizer from scratch."""
    
    def __init__(self, max_features: int = None):
        self.vocab = {}
        self.idf = {}
        self.max_features = max_features
    
    def fit(self, documents: List[str]):
        """Compute vocabulary and IDF values."""
        N = len(documents)
        doc_freq = Counter()  # How many docs contain each word
        
        # Count document frequencies
        for doc in documents:
            unique_words = set(doc.lower().split())
            for word in unique_words:
                doc_freq[word] += 1
        
        # Build vocabulary (optionally limit by frequency)
        if self.max_features:
            most_common = doc_freq.most_common(self.max_features)
            self.vocab = {word: idx for idx, (word, _) in enumerate(most_common)}
        else:
            self.vocab = {word: idx for idx, word in enumerate(sorted(doc_freq.keys()))}
        
        # Compute IDF: log(N / df(t)) with smoothing
        for word in self.vocab:
            self.idf[word] = math.log((N + 1) / (doc_freq[word] + 1)) + 1
        
        return self
    
    def transform(self, documents: List[str]) -> np.ndarray:
        """Convert documents to TF-IDF vectors."""
        vectors = np.zeros((len(documents), len(self.vocab)))
        
        for i, doc in enumerate(documents):
            words = doc.lower().split()
            word_counts = Counter(words)
            doc_len = len(words)
            
            for word, count in word_counts.items():
                if word in self.vocab:
                    # TF = count / doc_length
                    tf = count / doc_len
                    # TF-IDF = TF * IDF
                    vectors[i, self.vocab[word]] = tf * self.idf[word]
            
            # L2 normalize
            norm = np.linalg.norm(vectors[i])
            if norm > 0:
                vectors[i] /= norm
        
        return vectors
    
    def fit_transform(self, documents: List[str]) -> np.ndarray:
        return self.fit(documents).transform(documents)

# Test TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(corpus)

print(f"Shape: {X_tfidf.shape}")
print(f"\nIDF values (sample):")
for word in ['movie', 'amazing', 'terrible']:
    if word in tfidf.idf:
        print(f"  {word}: {tfidf.idf[word]:.3f}")

### 5.3 N-grams

In [ ]:
def get_ngrams(text: str, n: int) -> List[str]:
    """Extract n-grams from text."""
    words = text.lower().split()
    return ['_'.join(words[i:i+n]) for i in range(len(words) - n + 1)]

# Examples
text = "I love this movie"
print(f"Text: {text}")
print(f"Unigrams: {get_ngrams(text, 1)}")
print(f"Bigrams: {get_ngrams(text, 2)}")
print(f"Trigrams: {get_ngrams(text, 3)}")

## 6. PyTorch/Sklearn Implementation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer as SklearnTfidf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Sklearn TF-IDF
sklearn_tfidf = SklearnTfidf(ngram_range=(1, 2), max_features=100)
X_sklearn = sklearn_tfidf.fit_transform(corpus)

print(f"Sklearn TF-IDF shape: {X_sklearn.shape}")
print(f"Sample features: {sklearn_tfidf.get_feature_names_out()[:10]}")

# Quick classifier
clf = LogisticRegression()
clf.fit(X_sklearn, labels)

# Predict on new text
new_text = ["This film is absolutely amazing!"]
new_vec = sklearn_tfidf.transform(new_text)
pred = clf.predict(new_vec)
prob = clf.predict_proba(new_vec)

print(f"\nPrediction for '{new_text[0]}': {'Positive' if pred[0] == 1 else 'Negative'}")
print(f"Confidence: {prob[0][pred[0]]:.2%}")

## 7. 🔥 Real-World Usage

### Why TF-IDF is Still Relevant (2024)

| Use Case | Why TF-IDF |
|----------|------------|
| **Fast baseline** | Run it first! |
| **Interpretable** | Show which words matter |
| **Small data** | <1000 samples |
| **Search/Retrieval** | BM25 is TF-IDF variant |
| **Hybrid systems** | Combine with embeddings |

### Industry Example: Elasticsearch
```
Elasticsearch uses BM25 (TF-IDF variant) for text search.
Modern search: BM25 + Semantic Embeddings (hybrid)
```

## 8. Visualization

In [ ]:
import matplotlib.pyplot as plt

# Visualize TF-IDF for a document
doc_idx = 0
features = sklearn_tfidf.get_feature_names_out()
tfidf_scores = X_sklearn[doc_idx].toarray().flatten()

# Get top features
top_indices = tfidf_scores.argsort()[-10:][::-1]
top_features = [features[i] for i in top_indices]
top_scores = [tfidf_scores[i] for i in top_indices]

plt.figure(figsize=(10, 4))
plt.barh(top_features[::-1], top_scores[::-1])
plt.xlabel('TF-IDF Score')
plt.title(f'Top TF-IDF Features for: "{corpus[doc_idx][:30]}..."')
plt.tight_layout()
plt.show()

## 9. Common Mistakes

| Mistake | Fix |
|---------|-----|
| Not limiting vocab size | Use `max_features` |
| Ignoring OOV words | Track unknown word rate |
| Using on transformers | Transformers have their own tokenization |
| Forgetting normalization | Always L2 normalize |

## 10. Interview Questions

**Q1: What does IDF do? Why is it useful?**
<details><summary>Answer</summary>
IDF down-weights common words ("the", "is") and up-weights rare, informative words. A word in every document has low IDF; a word in one document has high IDF.
</details>

**Q2: How would you handle OOV words in TF-IDF?**
<details><summary>Answer</summary>
Options: (1) Ignore them, (2) Use a special UNK token, (3) Use subword/character n-grams, (4) Use embeddings for OOV
</details>

**Q3: When would you use TF-IDF over BERT?**
<details><summary>Answer</summary>
Small data (<1000), need explainability, strict latency (<10ms), baseline comparison, hybrid with semantic search
</details>

## 11. Summary

- **Bag of Words**: Simple word counts, sparse vectors
- **TF-IDF**: Weights words by importance (TF × IDF)
- **N-grams**: Capture local word patterns
- **Use as baseline** before trying deep learning
- **Still relevant** for search, small data, explainability

## 12. Exercises

1. Implement character-level TF-IDF
2. Build a spam classifier using TF-IDF + Logistic Regression
3. Compare unigrams, bigrams, and trigrams on sentiment task
4. Implement BM25 (TF-IDF variant used in search)

## 13. References

- [TF-IDF Wikipedia](https://en.wikipedia.org/wiki/Tf–idf)
- [BM25: The Next Generation of TF-IDF](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables)
- [Scikit-learn Text Features](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

---
**Next:** [Module 03: Word Embeddings](../03_word_embeddings/03_word_embeddings.ipynb)